# _How Do I Set Default Arguments to Make Function Calls Easier?_

## Calling functions with default arguments

We've seen this before when sorting a list of integers:

In [1]:
numbers = [3, 7, 1, 4, 2]
numbers

[3, 7, 1, 4, 2]

In [2]:
numbers.sort()

In [3]:
numbers

[1, 2, 3, 4, 7]

**By default**, the `sort()` method sorts the list in ascending order.
Consider the signature of the method:

In [4]:
help(numbers.sort)

Help on built-in function sort:

sort(*, key=None, reverse=False) method of builtins.list instance
    Sort the list in ascending order and return None.
    
    The sort is in-place (i.e. the list itself is modified) and stable (i.e. the
    order of two equal elements is maintained).
    
    If a key function is given, apply it once to each list item and sort them,
    ascending or descending, according to their function values.
    
    The reverse flag can be set to sort in descending order.



There are other parameters here: `key` and `reverse`. Both have default values. The parameter `reverse` is by default set to `False` such that by default the sorting is in ascending order.
If we want to sort the list in descending order:

In [5]:
numbers.sort(reverse=True)

In [6]:
numbers

[7, 4, 3, 2, 1]

## Defining functions with default arguments

Let's consider the following function that operates on an instance of `Task` class. It changes the status of a task to complete.
The serves one purpose take few arguments.

In [7]:
class Task:
    def __init__(self, title, description, urgency):
        self.title = title
        self.description = description
        self.urgency = urgency

In [8]:
def complete_task(task):
    task.status = "completed"
    print(f"{task.title}'s status: {task.status}")

In [9]:
task = Task("Homework", "Physics and math", 5)

In [10]:
complete_task(task)

Homework's status: completed


Now we want to update the function `complete_task()` to allow us adding a completion note on the task:

In [11]:
def complete_task(task, note):
    task.status = "completed"
    task.note = note
    print(f"{task.title}'s status: {task.status}; note: {task.note}")

However, most of the use cases of this function does not require adding a completion note:

In [12]:
task_1 = Task("Housechore", "mop", 1)
task_2 = Task("Programming", "review", 2)

In [13]:
complete_task(task_1, "great!")

Housechore's status: completed; note: great!


In [14]:
complete_task(task, "")

Homework's status: completed; note: 


In the end, we may end up repeating ourselves by passing an empty string everytime we call the function.
This is where setting a default argument comes in:

In [15]:
def complete_task(task, note=""):
    task.status = "completed"
    task.note = note
    print(f"{task.title}'s status: {task.status}; note: {task.note}")    

In [16]:
task_1 = Task("Housechore", "mop", 1)
task_2 = Task("Programming", "review", 2)

Now we can call the function with note as an argument:

In [17]:
complete_task(task_1, "Great!")

Housechore's status: completed; note: Great!


and without an argument:

In [18]:
complete_task(task_2)

Programming's status: completed; note: 


Following our development process, we went through the following:

- First iteration: a simple single-parameter function.
- Second iteration: Now I need to add a completion note, add a new parameter.
- Third iteration: I see that most of the times adding note is not necessary; the function is called with an empty string. Set the default to an empty string to avoid repetition in the most widely use cases.

At the third iteration we reach a sweet point of convenience and flexibility.
For the user of the function, convenience here means that they don't need to specify additional argument in most of use cases.
For the developer of the function, convenience here means avoiding user making mistake specifying unnecessary arguments.

In terms of flexibility, we add a new feature to the existing function without changing the old calling signature. All the previous calls that don't contain "note" argument are still valid.

## Avoiding the pitfall of setting default arguments for mutable parameters

This is one of the most famous pitfalls in Python.
Consider the following function:

In [19]:
def complete_task(task, grouped_tasks = []):
    task.status = "completed"
    grouped_tasks.append(task.title)

    return grouped_tasks

Here the idea of the function is besides setting the status of a task to complete, we would append the title of the task in a list.
If the function is called without the optional argument, an empty list is used instead.

In [20]:
task_0 = Task("Homework", "Physics and math", 5)
task_1 = Task("Fishing", "Fishing at the lake", 3)

If we call the function without the optional argument we should get a list with one element:

In [21]:
work_tasks = complete_task(task_0)
work_tasks

['Homework']

Indeed, let's call with another task, we should get a list with one element because the default is an empty list:

In [22]:
play_tasks = complete_task(task_1)
play_tasks

['Homework', 'Fishing']

Nope! We have a list of two elements. What happened?
Actually these two lists are actually the same object:

In [23]:
id(work_tasks), id(play_tasks)

(2585208489280, 2585208489280)

In [24]:
work_tasks is play_tasks

True

Why? **Python evaluates the function signature when it is defined.** Any mutable default arguments are created during evaluation and become part of the function.
If the function body modifies the mutable arguments, these modifications are carried from one call to another.
Actually, if a mutable argument is used, a particular object is used for that argument.

In [25]:
def append_task(task, tasks=[]):
    tasks.append(task)
    print(f"task: {tasks}; id: {id(tasks)}")

Let's check the default of this function:

In [26]:
append_task.__defaults__

([],)

which is a tuple whose first element is the empty list we set as the default. It has the following id:

In [27]:
id(append_task.__defaults__[0])

2585208666496

Suppose we call this function without the optional argument:

In [28]:
append_task("Homework")

task: ['Homework']; id: 2585208666496


In [29]:
append_task("Dishwashing")

task: ['Homework', 'Dishwashing']; id: 2585208666496


We keep modifying the same list between one function call to another. And indeed the default of this function has been altered:

In [30]:
append_task.__defaults__

(['Homework', 'Dishwashing'],)

The goal of the function is actually passing an optional mutable argument so that the body of the function can modify it when such an argument is passed. If not passed, then there should not be any mutable instance that got modified.
A better implementation is therefore:

In [31]:
def append_task(task, tasks=None):
    if tasks is None:
        tasks = []

    tasks.append(task)
    print(f"task: {tasks}; id: {id(tasks)}")

In [32]:
append_task("Homework")

task: ['Homework']; id: 2585208507392


In [33]:
append_task("Dishwashing")

task: ['Dishwashing']; id: 2585208710208


We have two different list instances everytime the function is called without the optional argument.
We can still pass the argument that kept the same instance from one call to another:

In [34]:
tasks = []
id(tasks)

2585208488064

In [35]:
append_task("Homework", tasks)

task: ['Homework']; id: 2585208488064


In [36]:
append_task("Dishwashing", tasks)

task: ['Homework', 'Dishwashing']; id: 2585208488064


## Challenges

Cory wants to show his students that the default arguments are evaluated when a function is defined not when it is called.

In [37]:
from datetime import datetime

In [38]:
print(datetime.today())
def my_function(timestamp = datetime.today()):
    return timestamp

2024-03-01 18:14:10.805946


In [39]:
print(my_function())

2024-03-01 18:14:10.805946


In [40]:
print(my_function())

2024-03-01 18:14:10.805946


In [41]:
my_function(datetime.today())

datetime.datetime(2024, 3, 1, 18, 14, 10, 864986)

In [42]:
my_function(datetime.today())

datetime.datetime(2024, 3, 1, 18, 14, 10, 912709)

In [43]:
my_function(datetime.today())

datetime.datetime(2024, 3, 1, 18, 14, 10, 930559)

Calling the function without the optional argument basically returns the same object that is created when the function definition is evaluated. The default argument remains the same from one call to another. Compare that with the case when we explicitly call the function with a new `datetime` instance; the outcome differs.